### ***4***

# LOGISTIC REGRESSION AND BGD






In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%reset-f

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy

In [5]:
path = '/content/drive/MyDrive/ML/data_q4_q5.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
for i in range(m):
  insts[i,-1] = (ord(insts[i,-1])-66)/11
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n = len(insts[0,:])-1 #no of features including f0 n=31
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std()


insts_mb = copy.deepcopy(insts)
insts_s = copy.deepcopy(insts)
np.random.shuffle(insts)
insts_tr = insts[0:int(m*0.7),:]
insts_te = insts[int(m*0.7):int(m*0.9),:]
insts_val = insts[int(m*0.9):m,:]
m_tr = len(insts_tr[:,0])

In [6]:
# hypothesis for logistic regression which is the prediction 
def hypothesis(w,insts,m,n):
  z = np.dot(insts[:,0:n],w.T)
  h = 1/(1+np.exp(-z.astype(np.float32))) #working
  return h

# cost function for logistic regression
def cost(h,insts,m):
  c = 0
  for i in range(m):
    c_te = (-insts_tr[i,-1]*np.log(h[i]) - (1- insts_tr[i,-1])*np.log(1 - h[i]))/m
    if not(np.isnan(c_te)) and not(math.isinf(c_te)):
      c = c+ c_te
  return c
# update of weight values for logistic regression
def update(w,alpha,insts,h,m,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = np.dot( (h[0:m]- insts[0:m,-1]),insts[0:m,i] )
  for i in range(n):
    w[i] = w[i] - alpha*d[i]

  return w
def update_stotastic(w,alpha,insts,h,index,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = (h[index]- insts[index,-1])*insts[index,i] 
  for i in range(n):
    w[i] = w[i] - alpha*d[i]

  return w

def performance(mat):
  accuracy = (mat[0,0]+mat[1,1])/(mat[0,0]+mat[1,1]+mat[0,1]+mat[1,0])
  sensitivity = (mat[1,1])/(mat[1,1]+mat[1,0])
  specificity = (mat[0,0])/(mat[0,1]+mat[0,0])
  print('accuracy :',end=' ')
  print(accuracy)
  print('sensitivity :',end=' ')
  print(sensitivity)
  print('specificity :',end=' ')
  print(specificity)

In [7]:
itr = 500
alpha = np.linspace(0.0001,0.001,100)
m_tr = len(insts_tr[:,0])
m_val = len(insts_val[:,0])
b_min = 10000
c = np.ones(len(alpha))
w_opt = np.zeros(n)  #stores the optimal weight value
j_opt = np.ones(itr) #stores the optimal j value list
min=0

In [8]:

for i in range(len(alpha)):
  w = np.random.rand(n)
  j_list = np.ones(itr)
  for j in range(itr):
    h = hypothesis(w,insts_tr,m_tr,n)
    w = update(w,alpha[i],insts_tr,h,m_tr,n)
    j_list[j] = cost(h,insts_tr,m_tr)
  h = hypothesis(w,insts_val,m_val,n)
  c[i] = cost(h,insts_val,m_val)
  if c[i] < b_min:
    min = i
    b_min = c[i]
    w_opt = w
    j_opt= j_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [9]:
m_te = len(insts_te[:,0])
h_te = hypothesis(w_opt,insts_te,m_te,n)
y_pre = np.round(h_te)
y_acu = insts_te[:,-1]

In [10]:

TRUNEG = 0
TP = 0
FN = 0
FP = 0
for i in range(m_te):
  if y_pre[i] == 0:
    if y_acu[i] == 0:
      TRUNEG = TRUNEG +1
    else:
      FP = FP +1
  else:
    if y_acu[i] == 1:
      TP = TP +1
    else:
      FN = FN +1
con_matrix = np.array([[TRUNEG,FP],[FN,TP]])
performance(con_matrix)
print('Confusion matrix :')
print(con_matrix)

accuracy : 0.956140350877193
sensitivity : 0.9230769230769231
specificity : 0.9838709677419355
Confusion matrix :
[[61  1]
 [ 4 48]]


# LOGISTIC REGRESSION AND MBG

In [11]:
np.random.shuffle(insts_mb)
insts_tr = insts_mb[0:int(m*0.7),:]
insts_te = insts_mb[int(m*0.7):int(m*0.9),:]
insts_val = insts_mb[int(m*0.9):m,:]
m_tr = len(insts_tr[:,0])

In [12]:
itr = 100
alpha = np.linspace(0.0001,0.001,10)
m_tr = len(insts_tr[:,0])
m_val = len(insts_val[:,0])
b_min = 10000
c = np.ones(len(alpha))
w_opt = np.zeros(n)  #stores the optimal weight value
j_opt = np.ones(itr) #stores the optimal j value list
min=0
b_size = 20

In [13]:
for i in range(len(alpha)):
  w = np.random.rand(n)
  j_list = np.ones(itr)
  for j in range(itr):
    np.random.shuffle(insts_tr)
    h = hypothesis(w,insts_tr,m_tr,n)
    w = update(w,alpha[i],insts_tr,h,b_size,n)
    j_list[j] = cost(h,insts_tr,m_tr)
  h = hypothesis(w,insts_val,m_val,n)
  c[i] = cost(h,insts_val,m_val)
  if c[i] < b_min:
    min = i
    b_min = c[i]
    w_opt = w
    j_opt= j_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [14]:
m_te = len(insts_te[:,0])
h_te = hypothesis(w_opt,insts_te,m_te,n)
y_pre = np.round(h_te)
y_acu = insts_te[:,-1]

In [15]:
TRUNEG = 0
TP = 0
FN = 0
FP = 0
for i in range(m_te):
  if y_pre[i] == 0:
    if y_acu[i] == 0:
      TRUNEG = TRUNEG +1
    else:
      FP = FP +1
  else:
    if y_acu[i] == 1:
      TP = TP +1
    else:
      FN = FN +1
con_matrix = np.array([[TRUNEG,FP],[FN,TP]])
performance(con_matrix)
print('Confusion matrix :')
print(con_matrix)

accuracy : 0.956140350877193
sensitivity : 0.9130434782608695
specificity : 0.9852941176470589
Confusion matrix :
[[67  1]
 [ 4 42]]


# LOGISTIC REGRESSION AND SGD

In [16]:
np.random.shuffle(insts_s)
insts_tr = insts_s[0:int(m*0.7),:]
insts_te = insts_s[int(m*0.7):int(m*0.9),:]
insts_val = insts_s[int(m*0.9):m,:]
m_tr = len(insts_tr[:,0])

In [17]:
itr = 500
alpha = np.linspace(0.0001,0.01,100)
m_tr = len(insts_tr[:,0])
m_val = len(insts_val[:,0])
b_min = 10000
c = np.ones(len(alpha))
w_opt = np.zeros(n)  #stores the optimal weight value
j_opt = np.ones(itr) #stores the optimal j value list
min=0

In [18]:
for i in range(len(alpha)):
  w = np.random.rand(n)
  j_list = np.ones(itr)
  for j in range(itr):
    index = np.random.randint(0,m_tr) # select a random index from the training data 
    h = hypothesis(w,insts_tr,m_tr,n)
    w = update_stotastic(w,alpha[i],insts_tr,h,index,n) # using the selected random instance to update the weight values
    j_list[j] = cost(h,insts_tr,m_tr)
  h = hypothesis(w,insts_val,m_val,n)
  c[i] = cost(h,insts_val,m_val)
  if c[i] < b_min:
    min = i
    b_min = c[i]
    w_opt = w
    j_opt= j_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [19]:
m_te = len(insts_te[:,0])
h_te = hypothesis(w_opt,insts_te,m_te,n)
y_pre = np.round(h_te)
y_acu = insts_te[:,-1]

In [20]:
TRUNEG = 0
TP = 0
FN = 0
FP = 0
for i in range(m_te):
  if y_pre[i] == 0:
    if y_acu[i] == 0:
      TRUNEG = TRUNEG +1
    else:
      FP = FP +1
  else:
    if y_acu[i] == 1:
      TP = TP +1
    else:
      FN = FN +1
con_matrix = np.array([[TRUNEG,FP],[FN,TP]])
performance(con_matrix)
print('Confusion matrix :')
print(con_matrix)

accuracy : 0.956140350877193
sensitivity : 0.9210526315789473
specificity : 0.9736842105263158
Confusion matrix :
[[74  2]
 [ 3 35]]


# LOGISTIC REGRESSION AND BGD + L2-NORM

In [21]:
%reset-f
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy

In [22]:
path = '/content/drive/MyDrive/ML/data_q4_q5.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
for i in range(m):
  insts[i,-1] = (ord(insts[i,-1])-66)/11
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n = len(insts[0,:])-1 #no of features including f0 n=31
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std()


insts_mb = copy.deepcopy(insts)
insts_s = copy.deepcopy(insts)
np.random.shuffle(insts)
insts_tr = insts[0:int(m*0.7),:]
insts_te = insts[int(m*0.7):int(m*0.9),:]
insts_val = insts[int(m*0.9):m,:]
m_tr = len(insts_tr[:,0])

In [23]:
# hypothesis for logistic regression which is the prediction 
def hypothesis_l2(w,insts,m,n):
  z = np.dot(insts[:,0:n],w.T)
  h = 1/(1+np.exp(-z.astype(np.float32))) #working
  return h

# cost function for logistic regression
def cost_l2(h,insts,lamb,m,w):
  c = 0
  for i in range(m):
    c_te = (-insts_tr[i,-1]*np.log(h[i]) - (1- insts_tr[i,-1])*np.log(1 - h[i]))/m
    if not(np.isnan(c_te)) and not(math.isinf(c_te)):
      c = c+ c_te
  return c + 0.5*lamb*(np.dot(w,w))
# update of weight values for logistic regression
def update_l2(w,alpha,lamb,insts,h,m,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = np.dot( (h[0:m]- insts[0:m,-1]),insts[0:m,i] ) 
   
  for i in range(n):
    w[i] = w[i]*(1-alpha*lamb) - alpha*d[i]

  return w
def update_stotastic_l2(w,alpha,lamb,insts,h,index,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = (h[index]- insts[index,-1])*insts[index,i] 
  for i in range(n):
    w[i] = w[i]*(1-alpha*lamb) - alpha*d[i]

  return w

def performance_l2(mat):
  accuracy = (mat[0,0]+mat[1,1])/(mat[0,0]+mat[1,1]+mat[0,1]+mat[1,0])
  sensitivity = (mat[1,1])/(mat[1,1]+mat[1,0])
  specificity = (mat[0,0])/(mat[0,1]+mat[0,0])
  print('accuracy :',end=' ')
  print(accuracy)
  print('sensitivity :',end=' ')
  print(sensitivity)
  print('specificity :',end=' ')
  print(specificity)

In [24]:
itr = 100
alpha = np.linspace(0.0001,0.001,10)
lamb = np.linspace(0,100,10)
m_tr = len(insts_tr[:,0])
m_val = len(insts_val[:,0])
b_min = 10000
c = np.ones(len(alpha))
w_opt = np.zeros(n)  #stores the optimal weight value
j_opt = np.ones(itr) #stores the optimal j value list
min=0

In [25]:
for i in range(len(lamb)):
  for a in alpha:
    w = np.random.rand(n)
    j_list = np.ones(itr)
    for j in range(itr):
      h = hypothesis_l2(w,insts_tr,m_tr,n)
      w = update_l2(w,a,lamb[i],insts_tr,h,m_tr,n)
      j_list[j] = cost_l2(h,insts_tr,lamb[i],m_tr,w)
    h = hypothesis_l2(w,insts_val,m_val,n)
    c[i] = cost_l2(h,insts_val,lamb[i],m_val,w)
    if c[i] < b_min:
      min = i
      b_min = c[i]
      w_opt = w
      j_opt= j_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [26]:
m_te = len(insts_te[:,0])
h_te = hypothesis_l2(w_opt,insts_te,m_te,n)
y_pre = np.round(h_te)
y_acu = insts_te[:,-1]

In [27]:
TRUNEG = 0
TP = 0
FN = 0
FP = 0
for i in range(m_te):
  if y_pre[i] == 0:
    if y_acu[i] == 0:
      TRUNEG = TRUNEG +1
    else:
      FP = FP +1
  else:
    if y_acu[i] == 1:
      TP = TP +1
    else:
      FN = FN +1
con_matrix = np.array([[TRUNEG,FP],[FN,TP]])
performance_l2(con_matrix)
print('Confusion matrix :')
print(con_matrix)

accuracy : 0.9210526315789473
sensitivity : 0.8653846153846154
specificity : 0.967741935483871
Confusion matrix :
[[60  2]
 [ 7 45]]


# LOGISTIC REGRESSION AND MBG + L2-NORM

In [28]:
np.random.shuffle(insts_mb)
insts_tr = insts_mb[0:int(m*0.7),:]
insts_te = insts_mb[int(m*0.7):int(m*0.9),:]
insts_val = insts_mb[int(m*0.9):m,:]
m_tr = len(insts_tr[:,0])

In [29]:
itr = 100
alpha = np.linspace(0.0001,0.001,10)
lamb = np.linspace(0,100,10)
m_tr = len(insts_tr[:,0])
m_val = len(insts_val[:,0])
b_min = 10000
c = np.ones(len(alpha))
w_opt = np.zeros(n)  #stores the optimal weight value
j_opt = np.ones(itr) #stores the optimal j value list
min=0
m_batch = 20

In [30]:
for i in range(len(lamb)):
  for a in alpha:
    w = np.random.rand(n)
    j_list = np.ones(itr)
    for j in range(itr):
      np.random.shuffle(insts_tr)
      h = hypothesis_l2(w,insts_tr,m_tr,n)
      w = update_l2(w,a,lamb[i],insts_tr,h,m_batch,n)
      j_list[j] = cost_l2(h,insts_tr,lamb[i],m_tr,w)
    h = hypothesis_l2(w,insts_val,m_val,n)
    c[i] = cost_l2(h,insts_val,lamb[i],m_val,w)
    if c[i] < b_min:
      min = i
      b_min = c[i]
      w_opt = w
      j_opt= j_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [31]:
m_te = len(insts_te[:,0])
h_te = hypothesis_l2(w_opt,insts_te,m_te,n)
y_pre = np.round(h_te)
y_acu = insts_te[:,-1]

In [32]:
TRUNEG = 0
TP = 0
FN = 0
FP = 0
for i in range(m_te):
  if y_pre[i] == 0:
    if y_acu[i] == 0:
      TRUNEG = TRUNEG +1
    else:
      FP = FP +1
  else:
    if y_acu[i] == 1:
      TP = TP +1
    else:
      FN = FN +1
con_matrix = np.array([[TRUNEG,FP],[FN,TP]])
performance_l2(con_matrix)
print('Confusion matrix :')
print(con_matrix)

accuracy : 0.8596491228070176
sensitivity : 0.7959183673469388
specificity : 0.9076923076923077
Confusion matrix :
[[59  6]
 [10 39]]


# LOGISTIC REGRESSION AND SGD + L2-NORM

In [33]:
np.random.shuffle(insts_s)
insts_tr = insts_s[0:int(m*0.7),:]
insts_te = insts_s[int(m*0.7):int(m*0.9),:]
insts_val = insts_s[int(m*0.9):m,:]
m_tr = len(insts_tr[:,0])

In [34]:
itr = 100
alpha = np.linspace(0.0001,0.001,10)
lamb = np.linspace(0,100,10)
m_tr = len(insts_tr[:,0])
m_val = len(insts_val[:,0])
b_min = 10000
c = np.ones(len(alpha))
w_opt = np.zeros(n)  #stores the optimal weight value
j_opt = np.ones(itr) #stores the optimal j value list
min=0

In [35]:
for i in range(len(lamb)):
  for a in alpha:
    w = np.random.rand(n)
    j_list = np.ones(itr)
    for j in range(itr):
      index = np.random.randint(m_tr)
      h = hypothesis_l2(w,insts_tr,m_tr,n)
      w = update_stotastic_l2(w,a,lamb[i],insts_tr,h,index,n)
      j_list[j] = cost_l2(h,insts_tr,lamb[i],m_tr,w)
    h = hypothesis_l2(w,insts_val,m_val,n)
    c[i] = cost_l2(h,insts_val,lamb[i],m_val,w)
    if c[i] < b_min:
      min = i
      b_min = c[i]
      w_opt = w
      j_opt= j_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [36]:
m_te = len(insts_te[:,0])
h_te = hypothesis_l2(w_opt,insts_te,m_te,n)
y_pre = np.round(h_te)
y_acu = insts_te[:,-1]

In [37]:
TRUNEG = 0
TP = 0
FN = 0
FP = 0
for i in range(m_te):
  if y_pre[i] == 0:
    if y_acu[i] == 0:
      TRUNEG = TRUNEG +1
    else:
      FP = FP +1
  else:
    if y_acu[i] == 1:
      TP = TP +1
    else:
      FN = FN +1
con_matrix = np.array([[TRUNEG,FP],[FN,TP]])
performance_l2(con_matrix)
print('Confusion matrix :')
print(con_matrix)

accuracy : 0.956140350877193
sensitivity : 0.9130434782608695
specificity : 0.9852941176470589
Confusion matrix :
[[67  1]
 [ 4 42]]


# LOGISTIC REGRESSION AND BGD + L1-NORM

In [38]:
%reset-f

In [39]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy

In [40]:
path = '/content/drive/MyDrive/ML/data_q4_q5.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
for i in range(m):
  insts[i,-1] = (ord(insts[i,-1])-66)/11
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n = len(insts[0,:])-1 #no of features including f0 n=31
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std()


insts_mb = copy.deepcopy(insts)
insts_s = copy.deepcopy(insts)
np.random.shuffle(insts)
insts_tr = insts[0:int(m*0.7),:]
insts_te = insts[int(m*0.7):int(m*0.9),:]
insts_val = insts[int(m*0.9):m,:]
m_tr = len(insts_tr[:,0])

In [41]:
# hypothesis for logistic regression which is the prediction 
def hypothesis_l1(w,insts,m,n):
  z = np.dot(insts[:,0:n],w.T)
  h = 1/(1+np.exp(-z.astype(np.float32))) #working
  return h

# cost function for logistic regression
def cost_l1(h,insts,lamb,m,w):
  c = 0
  for i in range(m):
    c_te = (-insts_tr[i,-1]*np.log(h[i]) - (1- insts_tr[i,-1])*np.log(1 - h[i]))/m
    if not(np.isnan(c_te)) and not(math.isinf(c_te)):
      c = c+ c_te
  w_abs = np.absolute(w)
  w_abs = np.sum(w_abs)
  return c + 0.5*lamb*(w_abs)
# update of weight values for logistic regression
def update_l1(w,alpha,lamb,insts,h,m,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = np.dot( (h[0:m]- insts[0:m,-1]),insts[0:m,i] ) 
   
  for i in range(n):
    w[i] = (w[i]-alpha*lamb*np.sign(w[i])) - alpha*d[i]

  return w
def update_stotastic_l1(w,alpha,lamb,insts,h,index,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = (h[index]- insts[index,-1])*insts[index,i] 
  for i in range(n):
    w[i] = (w[i]-alpha*lamb*np.sign(w[i])) - alpha*d[i]

  return w

def performance_l1(mat):
  accuracy = (mat[0,0]+mat[1,1])/(mat[0,0]+mat[1,1]+mat[0,1]+mat[1,0])
  sensitivity = (mat[1,1])/(mat[1,1]+mat[1,0])
  specificity = (mat[0,0])/(mat[0,1]+mat[0,0])
  print('accuracy :',end=' ')
  print(accuracy)
  print('sensitivity :',end=' ')
  print(sensitivity)
  print('specificity :',end=' ')
  print(specificity)

In [42]:
itr = 100
alpha = np.linspace(0.0001,0.001,10)
lamb = np.linspace(0,100,10)
m_tr = len(insts_tr[:,0])
m_val = len(insts_val[:,0])
b_min = 10000
c = np.ones(len(alpha))
w_opt = np.zeros(n)  #stores the optimal weight value
j_opt = np.ones(itr) #stores the optimal j value list
min=0

In [43]:
for i in range(len(lamb)):
  for a in alpha:
    w = np.random.rand(n)
    j_list = np.ones(itr)
    for j in range(itr):
      h = hypothesis_l1(w,insts_tr,m_tr,n)
      w = update_l1(w,a,lamb[i],insts_tr,h,m_tr,n)
      j_list[j] = cost_l1(h,insts_tr,lamb[i],m_tr,w)
    h = hypothesis_l1(w,insts_val,m_val,n)
    c[i] = cost_l1(h,insts_val,lamb[i],m_val,w)
    if c[i] < b_min:
      min = i
      b_min = c[i]
      w_opt = w
      j_opt= j_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [44]:
m_te = len(insts_te[:,0])
h_te = hypothesis_l1(w_opt,insts_te,m_te,n)
y_pre = np.round(h_te)
y_acu = insts_te[:,-1]

In [45]:
TRUNEG = 0
TP = 0
FN = 0
FP = 0
for i in range(m_te):
  if y_pre[i] == 0:
    if y_acu[i] == 0:
      TRUNEG = TRUNEG +1
    else:
      FP = FP +1
  else:
    if y_acu[i] == 1:
      TP = TP +1
    else:
      FN = FN +1
con_matrix = np.array([[TRUNEG,FP],[FN,TP]])
performance_l1(con_matrix)
print('Confusion matrix :')
print(con_matrix)

accuracy : 0.9736842105263158
sensitivity : 0.9545454545454546
specificity : 0.9857142857142858
Confusion matrix :
[[69  1]
 [ 2 42]]


# LOGISTIC REGRESSION AND MBG + L1-NORM

In [46]:
np.random.shuffle(insts_mb)
insts_tr = insts_mb[0:int(m*0.7),:]
insts_te = insts_mb[int(m*0.7):int(m*0.9),:]
insts_val = insts_mb[int(m*0.9):m,:]

In [47]:
itr = 100
alpha = np.linspace(0.01,0.1,10)
lamb = np.linspace(0,100,10)
m_tr = len(insts_tr[:,0])
m_val = len(insts_val[:,0])
b_min = 10000
c = np.ones(len(alpha))
w_opt = np.zeros(n)  #stores the optimal weight value
j_opt = np.ones(itr) #stores the optimal j value list
min=0
m_batch = 40

In [48]:
for i in range(len(lamb)):
  for a in alpha:
    w = np.random.rand(n)
    j_list = np.ones(itr)
    for j in range(itr):
      np.random.shuffle(insts_tr)
      h = hypothesis_l1(w,insts_tr,m_tr,n)
      w = update_l1(w,a,lamb[i],insts_tr,h,m_batch,n)
      j_list[j] = cost_l1(h,insts_tr,lamb[i],m_tr,w)
    h = hypothesis_l1(w,insts_val,m_val,n)
    c[i] = cost_l1(h,insts_val,lamb[i],m_val,w)
    if c[i] < b_min:
      min = i
      b_min = c[i]
      w_opt = w
      j_opt= j_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in multiply
  # This is added back by InteractiveShellApp.init_path()


In [49]:
m_te = len(insts_te[:,0])
h_te = hypothesis_l1(w_opt,insts_te,m_te,n)
y_pre = np.round(h_te)
y_acu = insts_te[:,-1]

In [50]:
TRUNEG = 0
TP = 0
FN = 0
FP = 0
for i in range(m_te):
  if y_pre[i] == 0:
    if y_acu[i] == 0:
      TRUNEG = TRUNEG +1
    else:
      FP = FP +1
  else:
    if y_acu[i] == 1:
      TP = TP +1
    else:
      FN = FN +1
con_matrix = np.array([[TRUNEG,FP],[FN,TP]])
performance_l1(con_matrix)
print('Confusion matrix :')
print(con_matrix)

accuracy : 0.9824561403508771
sensitivity : 0.9803921568627451
specificity : 0.9841269841269841
Confusion matrix :
[[62  1]
 [ 1 50]]


# LOGISTIC REGRESSION AND SGD + L1-NORM

In [51]:
np.random.shuffle(insts_s)
insts_tr = insts_s[0:int(m*0.7),:]
insts_te = insts_s[int(m*0.7):int(m*0.9),:]
insts_val = insts_s[int(m*0.9):m,:]

In [52]:
itr = 100
alpha = np.linspace(0.01,0.1,10)
lamb = np.linspace(0,100,10)
m_tr = len(insts_tr[:,0])
m_val = len(insts_val[:,0])
b_min = 10000
c = np.ones(len(alpha))
w_opt = np.zeros(n)  #stores the optimal weight value
j_opt = np.ones(itr) #stores the optimal j value list
min=0

In [53]:
for i in range(len(lamb)):
  for a in alpha:
    w = np.random.rand(n)
    j_list = np.ones(itr)
    for j in range(itr):
      index = np.random.randint(0,m_tr)
      h = hypothesis_l1(w,insts_tr,m_tr,n)
      w = update_stotastic_l1(w,a,lamb[i],insts_tr,h,index,n)
      j_list[j] = cost_l1(h,insts_tr,lamb[i],m_tr,w)
    h = hypothesis_l1(w,insts_val,m_val,n)
    c[i] = cost_l1(h,insts_val,lamb[i],m_val,w)
    if c[i] < b_min:
      min = i
      b_min = c[i]
      w_opt = w
      j_opt= j_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in multiply
  # This is added back by InteractiveShellApp.init_path()


In [54]:
m_te = len(insts_te[:,0])
h_te = hypothesis_l1(w_opt,insts_te,m_te,n)
y_pre = np.round(h_te)
y_acu = insts_te[:,-1]

In [55]:
TRUNEG = 0
TP = 0
FN = 0
FP = 0
for i in range(m_te):
  if y_pre[i] == 0:
    if y_acu[i] == 0:
      TRUNEG = TRUNEG +1
    else:
      FP = FP +1
  else:
    if y_acu[i] == 1:
      TP = TP +1
    else:
      FN = FN +1
con_matrix = np.array([[TRUNEG,FP],[FN,TP]])
performance_l1(con_matrix)
print('Confusion matrix :')
print(con_matrix)

accuracy : 0.9824561403508771
sensitivity : 0.9545454545454546
specificity : 1.0
Confusion matrix :
[[70  0]
 [ 2 42]]
